In [1]:
import sys
import os
from google.colab import drive

drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/rag/models'

sys.path.append(model_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install langchain_community
!pip install rank_bm25
!pip install faiss-cpu
!pip install langchain_core
!pip install langchain_openai
!pip install chromadb
!pip install langchain_chroma
!pip install langchain_openai
!pip install langchain_core
!pip install langchain_core
!pip install langchain_classic

In [3]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
import chromadb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

llm = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.01,
    max_tokens = 2000,
    openai_api_key = userdata.get('OPENAI_API_KEY')
)

db_path = "/content/drive/MyDrive/rag/chromadb_backup"
client = chromadb.PersistentClient(path=db_path)
collection_name = "tax_law"

try:
  collection = client.get_collection(name=collection_name)
except Exception:
  raise RuntimeError(f"Collection {collection_name} not found")

embedding_model = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embedding_model, openai_api_key = userdata.get("OPENAI_API_KEY"))

vectorstore = Chroma(client = client, collection_name=collection_name, embedding_function=embeddings)


## version3

In [4]:
from V3_ensemble import Pipeline, Reference

In [10]:
query = "조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."

pipeline = Pipeline(query, llm, vectorstore)
print(Reference().generate_reference(pipeline))

네, 결손금이 발생한 연도에 중소기업에 해당하면 결손금 소급공제 환급신청이 가능하며, 직전 사업연도에 중소기업이 아니더라도 환급이 인정됩니다 [[0]] [[3]].

[근거 법령 및 본문]법인세법 72조 ①항: 중소기업에 해당하는 내국법인은 각 사업연도에 결손금이 발생한 경우 대통령령으로 정하는 직전 사업연도의 법인세액(이하 이 조에서 "직전 사업연도의 법인세액"이라 한다)을 한도로 제1호의 금액에서 제2호의 금액을 차감한 금액을 환급 신청할 수 있다. <개정 2018.12.24>
법인세법 시행령 10조 ③항 3호: 법 제72조제1항 및 「조세특례제한법」 제8조의4에 따라 공제받은 결손금


In [6]:
from datasets import Dataset
from google.colab import userdata

try:
  from ragas import evaluate
except ImportError:
  !pip install ragas
  from ragas import evaluate

try:
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
except ImportError:
  !pip install ragas
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/tmp/ipython-input-2012395113.py:11: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
/tmp/ipython-input-2012395113.py:11: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics

In [7]:
import json
from IPython.display import display
import pandas as pd

file_path = ['/content/drive/MyDrive/rag/datasets/test_법인세법.json', '/content/drive/MyDrive/rag/datasets/test_vat.json', '/content/drive/MyDrive/rag/datasets/test_소득세.json', '/content/drive/MyDrive/rag/datasets/test_양도소득세.json', '/content/drive/MyDrive/rag/datasets/test_상증세.json']
category = ['법인세법', 'vat', '소득세', '양도소득세', '상증세']
final_result = []

for path, law_name in zip(file_path, category):

  try:
    with open(path, 'r', encoding='utf-8') as f:
      test_data = json.load(f)

    questions = [item['query'] for item in test_data]
    ground_truths = [item['answer'] for item in test_data]

    answers = []
    contexts = []

    for query in questions:
      try:
        answer, _, reranked_docs = Pipeline(query,llm, vectorstore).generate_answer()
        answers.append(answer)
        contexts.append([doc.page_content for doc in reranked_docs])
      except Exception as e:
        print(f"Error on query: {e}")
        answers.append("Error")
        contexts.append([])

    data = {
        "user_input": questions,
        "response": answers,
        "retrieved_contexts": contexts,
        "reference": ground_truths
    }

    dataset = Dataset.from_dict(data)

    llm_ragas = LangchainLLMWrapper(llm)
    embedding_ragas = LangchainEmbeddingsWrapper(embeddings)

    result = evaluate(dataset=dataset,
                      metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
                      llm=llm_ragas,
                      embeddings=embedding_ragas)

    df = result.to_pandas()
    df['law_name'] = law_name
    final_result.append(df)
    display(df)


  except Exception as e:
    print(f"{law_name} 평가 중 치명적 오류 발생: {e}")


  if final_result:
    final_report = pd.concat(final_result, ignore_index=True)
    summary = final_report.groupby('law_name')[['faithfulness', 'answer_relevancy', 'context_recall', 'context_precision']].mean()
    display(summary)

/tmp/ipython-input-371096220.py:40: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-371096220.py:41: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,law_name
0,조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직...,"[제72조(중소기업의 결손금 소급공제에 따른 환급), ②법 제13조제1항제1호에 따...","결손금이 발생한 연도에 중소기업에 해당되면 환급이 가능하며, 직전 사업연도에 중소기...",결손금 소급공제에 의한 환급 적용 대상 법인은 조세특례제한법 시행령 제2조의 규정에...,0.500000,0.596515,1.0,1.000000,법인세법
1,접대비 정규영수증 수취시 3만원 초과 접대비에 대하여 임· 직원의 신용카드를 사용하...,"[제41조(기업업무추진비의 신용카드등의 사용), 1. 「여신전문금융업법」에 의한 신...",임직원의 신용카드 사용으로 수취한 3만원 초과 접대비는 접대비로 인정됩니다.,경조사비 외 3만원 초과 접대비 지출증빙 중 신용카드매출전표는 법인명의의 신용카드 ...,0.000000,0.756624,1.0,1.000000,법인세법
2,법인이 장기 미회수 매출채권에 대하여 일정부분을 감면해주고 현금으로 회수하기로 합의...,[① 각 사업연도의 소득에 대하여 법과 다른 법률에 따라 법인세를 면제받거나 감면받...,감면해준 채무금액은 법인세법 시행령 제23조제1항에 따라 감가상각비로 손금에 산입할...,법인이 약정에 의하여 채권의 전부 또는 일부를 포기하는 경우 대손금으로 보지 아니하...,0.333333,0.513282,1.0,0.796190,법인세법
3,부동산매매업을 주업으로 하는 법인입니다. 부동산매매업 법인이 유예기간 경과 후 부동...,[1. 영 제49조제1항제1호 가목에 해당하는 부동산 : 당해 부동산을 업무에 직접...,"유예기간이 경과한 후 부동산을 양도하는 경우, 업무와 관련이 없는 것으로 보는 기간...",부동산매매업을 주업으로 하는 내국법인이 취득한 매매용 부동산을 법인세법 시행규칙 제...,1.000000,0.766914,1.0,1.000000,법인세법
4,A법인과 B법인은 주주가 가족으로 구성된 법인입니다. A법인은 사업이 잘되고 있으나...,"[3. 「법인세법」에 따라 상여로 처분된 금액, 다. 사양비, ① 손금은 자본 또는...","대여금에 대한 미수이자는 대손처리하지 않고 사외유출로 처리하는 것이 적절하며, 대표...",법인이 특수관계인에게 금전을 대여하는 경우 특수관계가 소멸할 때까지 회수되지 아니한...,0.000000,0.179618,1.0,0.759524,법인세법
5,법인세 신고 후 법인통장으로 입금된 매출누락이 발견되어 수정신고를 하는 경우 소득처...,"[④내국법인이 「국세기본법」 제45조의 수정신고기한내에 매출누락, 가공경비 등 부당...",내국법인이 수정신고기한 내에 매출누락을 회수하고 세무조정으로 익금에 산입하는 경우의...,법인의 각 사업연도 소득금액 계산상 매출누락이 확인되는 경우 매출누락액이 실제 법인...,1.000000,0.446274,1.0,1.000000,법인세법
6,유형자산의 재평가가 기업회계기준에 의해 허용된 바 토지재평가에 따른 평가차익이 손익...,"[1. 자산의 평가이익. 다만, 제42조제1항 각 호에 따른 평가로 인하여 발생하는...",토지재평가에 따른 평가차익은 법인세법 제42조제1항 각 호에 따른 평가로 인하여 발...,자산을 평가하면서 발생하는 평가차손익은 「법인세법」 제18조 및 제22조에서 규정하...,0.500000,0.719427,1.0,1.000000,법인세법
7,제조업을 영위하는 중소기업이 중소기업에 대한 특별세액감면을 적용함에 있어 국고보조금...,"[1. 각 사업연도의 소득에 대한 세액 감면(면제를 포함한다), ①제95조에 따른 ...","국고보조금은 감면소득에 해당하지 않으며, 관련 법령에서 별도로 규정되어 있지 않기 ...",국고보조금은 제조업 등 감면업종을 영위하는 사업에서 발생한 소득이 아니므로 제조업소...,0.333333,0.463544,1.0,0.873413,법인세법
8,대표이사는 현재 무주택자로 전세금 때문에 퇴직금 중간정산을 받으려고 합니다. 대표이...,[3. 「근로자퇴직급여 보장법」 제8조제2항에 따라 퇴직급여를 중간정산(종전에 퇴직...,"대표이사가 근로자퇴직급여보장법 시행령의 중간정산 사유에 해당한다면, 퇴직금 중간정산...",임원의 전세자금 마련은 중간정산 사유에 해당되지 않는 것입니다. 법인의 임원에 대한...,0.000000,0.301040,1.0,0.873413,법인세법
9,"법인이 기부금 단체인 생활복지단체 등에 현무롤 기부시, 법정기부금, 지정기부금에 따...",[④법인이 법 제24조에 따라 기부금을 지출한 때에는 법 제24조제2항제1호에 따른...,"법인은 법정기부금, 지정기부금에 따라 영수증을 시가 또는 장부가액으로 분리하여 받아...",기부금을 금전외의 자산으로 제공한 경우로서 지정기부금(특수관계인이 아닌 자에게 기부...,1.000000,0.585998,1.0,0.696190,법인세법


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.466667,0.532924,1.0,0.899873


/tmp/ipython-input-371096220.py:40: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-371096220.py:41: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


...

,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
vat,0.325000,0.719449,1.0,0.857619
법인세법,0.466667,0.532924,1.0,0.899873
소득세,0.683333,0.399209,0.8,0.730258
양도소득세,0.640000,0.769130,1.0,0.845754


/tmp/ipython-input-371096220.py:40: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-371096220.py:41: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,law_name
0,혼인·출산 증여재산공제 적용 후 이혼한 경우에도 당초 공제가 유지되나요?,[⑤ 제1항부터 제3항까지의 규정을 적용할 때 과세기간 종료일 이전에 혼인ㆍ이혼ㆍ별...,"이혼한 경우에는 혼인ㆍ출산 증여재산공제의 적용이 유지되지 않으며, 이혼 사유 발생 ...",증여받은 후 2년 이내에 혼인 신고하여 혼인·출산 증여재산공제를 적용받은 거주자가 ...,1.000000,0.534713,1.0,0.642857,상증세
1,금융재산을 사전증여받은 경우 상속세 과세시 금융재산상속공제 적용 여부?,"[② 영 제158조제4항제2호에서 ""재정경제부령으로 정하는 금융재산""이란 「금융실명...",금융재산을 사전증여받은 경우 상속세 과세시 금융재산상속공제는 적용되지 않습니다.,금융재산을 사전증여받은 경우 상속세 과세시 금융재산상속공제 적용 안됨. ｢상속세 ...,0.000000,0.340451,1.0,1.000000,상증세
2,"일반적으로 부모나 배우자, 친족으로부터 증여받은 경우 증여재산공제 금액은?",[1. 거주자가 그 배우자 또는 직계존비속으로부터 증여받은 자산에 대한 증여세 산출...,"일반적으로 부모나 배우자, 친족으로부터 증여받은 경우 증여재산공제 금액은 증여세 과...",1. 배우자로부터 증여를 받은 경우 : 6억원 2. 직계존속[수증자의 직계존속과 혼...,0.000000,0.284549,1.0,0.902857,상증세
3,보험료 납부자와 보험금 수령인이 다른 보험금에 대하여 증여세가 과세되는지요?,"[3. 계약변경에 따른 저축성보험계약의 계약자, 피보험자 및 수익자가 같은 사람일 ...","보험료 납부자와 보험금 수령인이 다른 경우, 보험금이 증여로 간주될 수 있어 증여세...",생명보험 또는 손해보험에 있어서 보험금수령인과 보험료 납부자가 다른 경우에는 보험사...,0.333333,0.826426,1.0,0.629524,상증세
4,이혼시 위자료 또는 재산분할에 의하여 취득한 재산에 대하여 증여세가 과세되는지요?,"[1. 「상속세 및 증여세법」 제3조의2제2항, 제33조부터 제39조까지, 제39조...","이혼시 위자료 또는 재산분할에 의하여 취득한 재산은 증여세 과세 대상이 아니며, 관...",이혼 등에 의하여 정신적 또는 재산상 손해배상의 대가로 받는 위자료는 조세포탈의 목...,0.000000,0.938283,1.0,0.000000,상증세


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
vat,0.325000,0.719449,1.0,0.857619
법인세법,0.466667,0.532924,1.0,0.899873
상증세,0.266667,0.584884,1.0,0.635048
소득세,0.683333,0.399209,0.8,0.730258
양도소득세,0.640000,0.769130,1.0,0.845754


In [18]:
import json
from IPython.display import display
import pandas as pd

file_path = ['/content/drive/MyDrive/rag/datasets/test_법인세법.json', '/content/drive/MyDrive/rag/datasets/test_vat.json', '/content/drive/MyDrive/rag/datasets/test_소득세.json', '/content/drive/MyDrive/rag/datasets/test_양도소득세.json', '/content/drive/MyDrive/rag/datasets/test_상증세.json']
category = ['법인세법', 'vat', '소득세', '양도소득세', '상증세']
final_result = []

for path, law_name in zip(file_path, category):

  try:
    with open(path, 'r', encoding='utf-8') as f:
      test_data = json.load(f)

    questions = [item['query'] for item in test_data]
    ground_truths = [item['answer'] for item in test_data]

    answers = []
    contexts = []

    for query in questions:
      try:
        answer, _, reranked_docs = Pipeline(query,llm, vectorstore).generate_answer()
        answers.append(answer)
        contexts.append([doc.page_content for doc in reranked_docs])
      except Exception as e:
        print(f"Error on query: {e}")
        answers.append("Error")
        contexts.append([])

    data = {
        "user_input": questions,
        "response": answers,
        "retrieved_contexts": contexts,
        "reference": ground_truths
    }

    dataset = Dataset.from_dict(data)

    llm_ragas = LangchainLLMWrapper(llm)
    embedding_ragas = LangchainEmbeddingsWrapper(embeddings)

    result = evaluate(dataset=dataset,
                      metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
                      llm=llm_ragas,
                      embeddings=embedding_ragas)

    df = result.to_pandas()
    df['law_name'] = law_name
    final_result.append(df)
    display(df)


  except Exception as e:
    print(f"{law_name} 평가 중 치명적 오류 발생: {e}")


  if final_result:
    final_report = pd.concat(final_result, ignore_index=True)
    summary = final_report.groupby('law_name')[['faithfulness', 'answer_relevancy', 'context_recall', 'context_precision']].mean()
    display(summary)

/tmp/ipython-input-371096220.py:40: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-371096220.py:41: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,law_name
0,조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직...,"[제72조(중소기업의 결손금 소급공제에 따른 환급), ②법 제13조제1항제1호에 따...","네, 결손금이 발생한 연도에 중소기업에 해당되었다면, 결손금 소급공제 환급신청을 통...",결손금 소급공제에 의한 환급 적용 대상 법인은 조세특례제한법 시행령 제2조의 규정에...,0.500000,0.361107,1.000000,1.000000,법인세법
1,접대비 정규영수증 수취시 3만원 초과 접대비에 대하여 임· 직원의 신용카드를 사용하...,"[제41조(기업업무추진비의 신용카드등의 사용), 1. 자산, 1. 상품, 1. 곡류...",임직원의 신용카드 사용으로 수취한 3만원 초과 접대비는 접대비로 인정됩니다.,경조사비 외 3만원 초과 접대비 지출증빙 중 신용카드매출전표는 법인명의의 신용카드 ...,0.000000,0.756580,1.000000,0.948413,법인세법
2,법인이 장기 미회수 매출채권에 대하여 일정부분을 감면해주고 현금으로 회수하기로 합의...,[① 각 사업연도의 소득에 대하여 법과 다른 법률에 따라 법인세를 면제받거나 감면받...,"감면해준 채무금액은 법인세법 시행령 제3호에 따라 손금에 산입할 수 있으며, 다만,...",법인이 약정에 의하여 채권의 전부 또는 일부를 포기하는 경우 대손금으로 보지 아니하...,0.000000,0.447499,1.000000,0.796190,법인세법
3,부동산매매업을 주업으로 하는 법인입니다. 부동산매매업 법인이 유예기간 경과 후 부동...,"[가. 법인의 업무에 직접 사용하지 아니하는 부동산. 다만, 재정경제부령이 정하는 ...","부동산을 양도하는 경우, 유예기간이 경과한 후부터 업무와 관련이 없는 부동산으로 간...",부동산매매업을 주업으로 하는 내국법인이 취득한 매매용 부동산을 법인세법 시행규칙 제...,1.000000,0.304735,1.000000,1.000000,법인세법
4,A법인과 B법인은 주주가 가족으로 구성된 법인입니다. A법인은 사업이 잘되고 있으나...,[②제104조제2항에 따라 결정된 과세표준과 법인의 재무상태표상의 당기순이익과의 차...,"이 경우, 대손처리하지 않고 사외유출로 처리하는 것이 맞으며, 대표이사 상여처분이 ...",법인이 특수관계인에게 금전을 대여하는 경우 특수관계가 소멸할 때까지 회수되지 아니한...,0.500000,0.225163,1.000000,0.948413,법인세법
5,법인세 신고 후 법인통장으로 입금된 매출누락이 발견되어 수정신고를 하는 경우 소득처...,"[④내국법인이 「국세기본법」 제45조의 수정신고기한내에 매출누락, 가공경비 등 부당...","법인세 신고 후 매출누락이 발견되어 수정신고를 하는 경우, 해당 금액은 사내유보로 ...",법인의 각 사업연도 소득금액 계산상 매출누락이 확인되는 경우 매출누락액이 실제 법인...,1.000000,0.840087,1.000000,1.000000,법인세법
6,유형자산의 재평가가 기업회계기준에 의해 허용된 바 토지재평가에 따른 평가차익이 손익...,"[1. 자산의 평가이익. 다만, 제42조제1항 각 호에 따른 평가로 인하여 발생하는...",토지재평가에 따른 평가차익은 법인세법상 평가이익으로 간주되어 손익으로 인정되지 않으...,자산을 평가하면서 발생하는 평가차손익은 「법인세법」 제18조 및 제22조에서 규정하...,0.333333,0.743698,1.000000,1.000000,법인세법
7,제조업을 영위하는 중소기업이 중소기업에 대한 특별세액감면을 적용함에 있어 국고보조금...,"[1. 각 사업연도의 소득에 대한 세액 감면(면제를 포함한다), ① 법 및 「조세특...","국고보조금이 감면소득에 해당하는지 여부는 제공된 자료에서 명확히 언급되지 않으므로,...",국고보조금은 제조업 등 감면업종을 영위하는 사업에서 발생한 소득이 아니므로 제조업소...,0.666667,0.000000,1.000000,1.000000,법인세법
8,대표이사는 현재 무주택자로 전세금 때문에 퇴직금 중간정산을 받으려고 합니다. 대표이...,[3. 「근로자퇴직급여 보장법」 제8조제2항에 따라 퇴직급여를 중간정산(종전에 퇴직...,"네, 대표이사도 근로자퇴직급여보장법 시행령의 중간정산 사유에 해당한다면 퇴직금 중간...",임원의 전세자금 마련은 중간정산 사유에 해당되지 않는 것입니다. 법인의 임원에 대한...,0.500000,0.622133,0.666667,0.817857,법인세법
9,"법인이 기부금 단체인 생활복지단체 등에 현무롤 기부시, 법정기부금, 지정기부금에 따...",[④법 제24조의 규정에 의하여 기부금을 지출한 법인이 손금산입을 하고자 하는 경우...,"법인은 기부금 단체에 현물 기부 시, 법정기부금과 지정기부금에 따라 시가 또는 장부...",기부금을 금전외의 자산으로 제공한 경우로서 지정기부금(특수관계인이 아닌 자에게 기부...,0.500000,0.680299,1.000000,0.562857,법인세법


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.5,0.49813,0.966667,0.907373


/tmp/ipython-input-371096220.py:40: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-371096220.py:41: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


...

,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
vat,0.400000,0.578404,1.000000,0.834270
법인세법,0.500000,0.498130,0.966667,0.907373
소득세,0.483333,0.436323,0.800000,0.613869
양도소득세,0.550000,0.388603,1.000000,0.823825


/tmp/ipython-input-371096220.py:40: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-371096220.py:41: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,law_name
0,혼인·출산 증여재산공제 적용 후 이혼한 경우에도 당초 공제가 유지되나요?,[⑤ 제1항부터 제3항까지의 규정을 적용할 때 과세기간 종료일 이전에 혼인ㆍ이혼ㆍ별...,이혼한 경우에는 혼인ㆍ이혼 사유로 인한 공제 적용이 종료되므로 당초 공제는 유지되지...,증여받은 후 2년 이내에 혼인 신고하여 혼인·출산 증여재산공제를 적용받은 거주자가 ...,1.000000,0.487417,1.0,1.000000,상증세
1,금융재산을 사전증여받은 경우 상속세 과세시 금융재산상속공제 적용 여부?,"[1. 공익단체가 「상속세 및 증여세법」 제48조제2항, 제3항, 제8항부터 제11...","금융재산을 사전증여받은 경우, 상속세 과세시 금융재산상속공제는 적용되지 않습니다.",금융재산을 사전증여받은 경우 상속세 과세시 금융재산상속공제 적용 안됨. ｢상속세 ...,0.000000,0.339868,1.0,1.000000,상증세
2,"일반적으로 부모나 배우자, 친족으로부터 증여받은 경우 증여재산공제 금액은?",[1. 거주자가 그 배우자 또는 직계존비속으로부터 증여받은 자산에 대한 증여세 산출...,"일반적으로 부모나 배우자, 친족으로부터 증여받은 경우 증여재산공제 금액은 해당 증여...",1. 배우자로부터 증여를 받은 경우 : 6억원 2. 직계존속[수증자의 직계존속과 혼...,0.000000,0.298337,1.0,0.915079,상증세
3,보험료 납부자와 보험금 수령인이 다른 보험금에 대하여 증여세가 과세되는지요?,[3. 종업원이 계약자이거나 종업원 또는 그 배우자 및 그 밖의 가족을 수익자로 하...,"보험료 납부자와 보험금 수령인이 다른 보험금에 대하여 증여세가 과세될 수 있으며, ...",생명보험 또는 손해보험에 있어서 보험금수령인과 보험료 납부자가 다른 경우에는 보험사...,1.000000,0.964962,1.0,1.000000,상증세
4,이혼시 위자료 또는 재산분할에 의하여 취득한 재산에 대하여 증여세가 과세되는지요?,"[1. 「상속세 및 증여세법」 제3조의2제2항, 제33조부터 제39조까지, 제39조...","이혼시 위자료 또는 재산분할에 의하여 취득한 재산은 증여세 과세 대상이 아니며, 관...",이혼 등에 의하여 정신적 또는 재산상 손해배상의 대가로 받는 위자료는 조세포탈의 목...,0.333333,0.900862,0.0,0.000000,상증세


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
vat,0.400000,0.578404,1.000000,0.834270
법인세법,0.500000,0.498130,0.966667,0.907373
상증세,0.466667,0.598289,0.800000,0.783016
소득세,0.483333,0.436323,0.800000,0.613869
양도소득세,0.550000,0.388603,1.000000,0.823825


In [27]:
import json
from IPython.display import display
import pandas as pd

file_path = ['/content/drive/MyDrive/rag/datasets/test_법인세법.json', '/content/drive/MyDrive/rag/datasets/test_vat.json', '/content/drive/MyDrive/rag/datasets/test_소득세.json', '/content/drive/MyDrive/rag/datasets/test_양도소득세.json', '/content/drive/MyDrive/rag/datasets/test_상증세.json']
category = ['법인세법', 'vat', '소득세', '양도소득세', '상증세']
final_result = []

for path, law_name in zip(file_path, category):

  try:
    with open(path, 'r', encoding='utf-8') as f:
      test_data = json.load(f)

    questions = [item['query'] for item in test_data]
    ground_truths = [item['answer'] for item in test_data]

    answers = []
    contexts = []

    for query in questions:
      try:
        answer, _, reranked_docs = Pipeline(query,llm, vectorstore).generate_answer()
        answers.append(answer)
        contexts.append([doc.page_content for doc in reranked_docs])
      except Exception as e:
        print(f"Error on query: {e}")
        answers.append("Error")
        contexts.append([])

    data = {
        "user_input": questions,
        "response": answers,
        "retrieved_contexts": contexts,
        "reference": ground_truths
    }

    dataset = Dataset.from_dict(data)

    llm_ragas = LangchainLLMWrapper(llm)
    embedding_ragas = LangchainEmbeddingsWrapper(embeddings)

    result = evaluate(dataset=dataset,
                      metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
                      llm=llm_ragas,
                      embeddings=embedding_ragas)

    df = result.to_pandas()
    df['law_name'] = law_name
    final_result.append(df)
    display(df)


  except Exception as e:
    print(f"{law_name} 평가 중 치명적 오류 발생: {e}")


  if final_result:
    final_report = pd.concat(final_result, ignore_index=True)
    summary = final_report.groupby('law_name')[['faithfulness', 'answer_relevancy', 'context_recall', 'context_precision']].mean()
    display(summary)

/tmp/ipython-input-371096220.py:40: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-371096220.py:41: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,law_name
0,조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직...,[① 중소기업에 해당하는 내국법인은 각 사업연도에 결손금이 발생한 경우 대통령령으로...,"네, 결손금이 발생한 연도에 중소기업에 해당하면 환급이 가능합니다.",결손금 소급공제에 의한 환급 적용 대상 법인은 조세특례제한법 시행령 제2조의 규정에...,1.000000,0.498977,1.000000,0.915079,법인세법
1,접대비 정규영수증 수취시 3만원 초과 접대비에 대하여 임· 직원의 신용카드를 사용하...,"[다. 소득공제대상이 되는 「조세특례제한법」 제126조의2제1항에 따른 신용카드, ...",임직원의 신용카드 사용으로 수취한 3만원 초과 접대비는 접대비로 인정됩니다.,경조사비 외 3만원 초과 접대비 지출증빙 중 신용카드매출전표는 법인명의의 신용카드 ...,0.000000,0.757242,1.000000,1.000000,법인세법
2,법인이 장기 미회수 매출채권에 대하여 일정부분을 감면해주고 현금으로 회수하기로 합의...,[3. 채무자의 인수거절ㆍ지급거절에 따라 채권금액의 회수가 불가능하거나 불가피하게 ...,감면해준 채무금액은 채권의 회수불능 또는 감액이 확인된 경우에 한해 손금으로 인정받...,법인이 약정에 의하여 채권의 전부 또는 일부를 포기하는 경우 대손금으로 보지 아니하...,1.000000,0.483700,1.000000,1.000000,법인세법
3,부동산매매업을 주업으로 하는 법인입니다. 부동산매매업 법인이 유예기간 경과 후 부동...,[1. 영 제49조제1항제1호 가목에 해당하는 부동산 : 당해 부동산을 업무에 직접...,"유예기간이 경과한 후 부동산을 양도하는 경우, 유예기간과 겹치는 기간을 제외한 이후...",부동산매매업을 주업으로 하는 내국법인이 취득한 매매용 부동산을 법인세법 시행규칙 제...,1.000000,0.302061,1.000000,1.000000,법인세법
4,A법인과 B법인은 주주가 가족으로 구성된 법인입니다. A법인은 사업이 잘되고 있으나...,"[제61조(대손충당금의 손금산입), 제19조의2(대손금의 손금불산입) 제19조의2(...","대여금에 대한 미수이자는 손금에 산입할 수 없으며, 대손처리하지 않고 사외유출로 처...",법인이 특수관계인에게 금전을 대여하는 경우 특수관계가 소멸할 때까지 회수되지 아니한...,0.000000,0.369336,1.000000,0.976190,법인세법
5,법인세 신고 후 법인통장으로 입금된 매출누락이 발견되어 수정신고를 하는 경우 소득처...,"[④내국법인이 「국세기본법」 제45조의 수정신고기한내에 매출누락, 가공경비 등 부당...",내국법인이 수정신고를 통해 매출누락 금액을 회수하고 세무조정으로 익금에 산입하는 경...,법인의 각 사업연도 소득금액 계산상 매출누락이 확인되는 경우 매출누락액이 실제 법인...,1.000000,0.241017,1.000000,1.000000,법인세법
6,유형자산의 재평가가 기업회계기준에 의해 허용된 바 토지재평가에 따른 평가차익이 손익...,"[1. 자산의 평가이익. 다만, 제42조제1항 각 호에 따른 평가로 인하여 발생하는...","토지재평가에 따른 평가차익은 손익으로 인정되지 않으며, 이는 법인세법 및 관련 규정...",자산을 평가하면서 발생하는 평가차손익은 「법인세법」 제18조 및 제22조에서 규정하...,0.000000,0.355995,0.250000,1.000000,법인세법
7,제조업을 영위하는 중소기업이 중소기업에 대한 특별세액감면을 적용함에 있어 국고보조금...,[⑧ 법 제36조제1항 및 제2항의 규정의 적용을 받고자 하는 내국법인은 법 제60...,"국고보조금은 감면소득에 해당하지 않으며, 중소기업에 대한 특별세액감면 적용 시 국고...",국고보조금은 제조업 등 감면업종을 영위하는 사업에서 발생한 소득이 아니므로 제조업소...,0.000000,0.509328,1.000000,1.000000,법인세법
8,대표이사는 현재 무주택자로 전세금 때문에 퇴직금 중간정산을 받으려고 합니다. 대표이...,[3. 「근로자퇴직급여 보장법」 제8조제2항에 따라 퇴직급여를 중간정산(종전에 퇴직...,"네, 대표이사도 근로자퇴직급여보장법 시행령의 중간정산 사유에 해당한다면 퇴직금 중간...",임원의 전세자금 마련은 중간정산 사유에 해당되지 않는 것입니다. 법인의 임원에 대한...,1.000000,0.621661,0.666667,0.667857,법인세법
9,"법인이 기부금 단체인 생활복지단체 등에 현무롤 기부시, 법정기부금, 지정기부금에 따...",[④법인이 법 제24조에 따라 기부금을 지출한 때에는 법 제24조제2항제1호에 따른...,"법인은 법정기부금과 지정기부금에 따라 영수증을 구분하여 발급할 필요가 있으며, 현물...",기부금을 금전외의 자산으로 제공한 경우로서 지정기부금(특수관계인이 아닌 자에게 기부...,0.333333,0.590814,1.000000,0.691667,법인세법


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.533333,0.473013,0.891667,0.925079


/tmp/ipython-input-371096220.py:40: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-371096220.py:41: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


...

,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
vat,0.433333,0.722168,0.950000,0.799817
법인세법,0.533333,0.473013,0.891667,0.925079
소득세,0.550000,0.582893,0.800000,0.694333
양도소득세,0.650000,0.577221,1.000000,0.871587


/tmp/ipython-input-371096220.py:40: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-371096220.py:41: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,law_name
0,혼인·출산 증여재산공제 적용 후 이혼한 경우에도 당초 공제가 유지되나요?,[⑤ 제1항부터 제3항까지의 규정을 적용할 때 과세기간 종료일 이전에 혼인ㆍ이혼ㆍ별...,"이혼한 경우에는 혼인ㆍ이혼 사유로 인한 공제 적용이 종료되므로, 당초 공제는 유지되...",증여받은 후 2년 이내에 혼인 신고하여 혼인·출산 증여재산공제를 적용받은 거주자가 ...,1.0,0.487318,1.0,0.642857,상증세
1,금융재산을 사전증여받은 경우 상속세 과세시 금융재산상속공제 적용 여부?,"[② 영 제158조제4항제2호에서 ""재정경제부령으로 정하는 금융재산""이란 「금융실명...",금융재산을 사전증여받은 경우 상속세 과세시 금융재산상속공제는 적용되지 않습니다.,금융재산을 사전증여받은 경우 상속세 과세시 금융재산상속공제 적용 안됨. ｢상속세 ...,0.0,0.345974,1.0,1.000000,상증세
2,"일반적으로 부모나 배우자, 친족으로부터 증여받은 경우 증여재산공제 금액은?",[1. 거주자가 그 배우자 또는 직계존비속으로부터 증여받은 자산에 대한 증여세 산출...,"일반적으로 부모나 배우자, 친족으로부터 증여받은 경우 증여재산공제 금액은 별도로 명...",1. 배우자로부터 증여를 받은 경우 : 6억원 2. 직계존속[수증자의 직계존속과 혼...,0.5,0.281935,1.0,0.876190,상증세
3,보험료 납부자와 보험금 수령인이 다른 보험금에 대하여 증여세가 과세되는지요?,[① 근로소득이 있는 거주자(일용근로자는 제외한다. 이하 이 조에서 같다)가 해당 ...,제공된 자료에서 보험료 납부자와 보험금 수령인이 다른 보험금에 대한 증여세 과세 여...,생명보험 또는 손해보험에 있어서 보험금수령인과 보험료 납부자가 다른 경우에는 보험사...,1.0,0.000000,1.0,0.525000,상증세
4,이혼시 위자료 또는 재산분할에 의하여 취득한 재산에 대하여 증여세가 과세되는지요?,"[1. 「상속세 및 증여세법」 제3조의2제2항, 제33조부터 제39조까지, 제39조...",이혼시 위자료 또는 재산분할에 의하여 취득한 재산에 대하여 증여세가 과세되지 않습니다.,이혼 등에 의하여 정신적 또는 재산상 손해배상의 대가로 받는 위자료는 조세포탈의 목...,0.0,0.901144,0.0,0.166667,상증세


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
vat,0.433333,0.722168,0.950000,0.799817
법인세법,0.533333,0.473013,0.891667,0.925079
상증세,0.500000,0.403274,0.800000,0.642143
소득세,0.550000,0.582893,0.800000,0.694333
양도소득세,0.650000,0.577221,1.000000,0.871587


## version2

In [8]:
from V2_multiquery import MultiQueryPipeline, Reference

In [9]:
query = "조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."

pipeline = MultiQueryPipeline(query, llm, vectorstore)
print(Reference().generate_reference(pipeline))

네, 결손금이 발생한 연도에 중소기업에 해당하는 경우에는 결손금 소급공제 환급신청이 가능하며, 직전 사업연도에 중소기업이 아니었더라도 환급이 가능합니다[[0]].

[근거 법령 및 본문]소득세법 85조 ①항: 「조세특례제한법」 제6조제1항에 따른 중소기업을 경영하는 거주자가 그 중소기업의 사업소득금액을 계산할 때 제45조제3항에서 규정하는 해당 과세기간의 이월결손금(부동산임대업에서 발생한 이월결손금은 제외한다. 이하 이 조에서 같다)이 발생한 경우에는 직전 과세기간의 그 중소기업의 사업소득에 부과된 종합소득 결정세액을 한도로 하여 대통령령으로 정하는 바에 따라 계산한 금액(이하 "결손금 소급공제세액"이라 한다)을 환급신청할 수 있다. 이 경우 소급공제한 이월결손금에 대해서 제45조제3항을 적용할 때에는 그 이월결손금을 공제받은 금액으로 ...
법인세법 시행령 10조 ②항: 법 제13조제1항제1호에 따라 결손금을 공제할 때에는 먼저 발생한 사업연도의 결손금부터 차례대로 공제한다. <개정 2016.2.12, 2019.2.12>


In [11]:
import typing
import pandas as pd
import json


def evaluation(version: str):
  file_path = ['/content/drive/MyDrive/rag/datasets/test_법인세법.json', '/content/drive/MyDrive/rag/datasets/test_vat.json', '/content/drive/MyDrive/rag/datasets/test_소득세.json', '/content/drive/MyDrive/rag/datasets/test_양도소득세.json', '/content/drive/MyDrive/rag/datasets/test_상증세.json']
  category = ['법인세법', 'vat', '소득세', '양도소득세', '상증세']
  final_result = []

  llm_ragas = LangchainLLMWrapper(llm)
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


  for path, law_name in zip(file_path, category):

    try:
      with open(path, 'r', encoding='utf-8') as f:
        test_data = json.load(f)

      questions = [item['query'] for item in test_data]
      ground_truths = [item['answer'] for item in test_data]

      answers = []
      contexts = []

      for query in questions:
        try:
          if version == "v3":
            pipeline = Pipeline(query,llm, vectorstore)

          elif version == "v2":
            pipeline = MultiQueryPipeline(query, llm, vectorstore)

          elif version == "v1":
            pipeline = ParentPipeline(query, llm, vectorstore)

          else:
            raise ValueError("Invalid version. Choose v2 or v3")

          answer, _, reranked_docs = pipeline.generate_answer()

          answers.append(answer)
          contexts.append([doc.page_content for doc in reranked_docs])

        except Exception as e:
          print(f"Error on query: {e}")
          continue

      data = {
          "user_input": questions,
          "response": answers,
          "retrieved_contexts": contexts,
          "reference": ground_truths
      }

      dataset = Dataset.from_dict(data)

      result = evaluate(dataset=dataset,
                        metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
                        llm=llm_ragas,
                        embeddings=embedding_ragas)

      df = result.to_pandas()
      df['law_name'] = law_name
      final_result.append(df)
      display(df)

    except Exception as e:
        print(f"{law_name} 평가 중 오류 발생: {e}")


    if final_result:
      final_report = pd.concat(final_result, ignore_index=True)
      summary = final_report.groupby('law_name')[['faithfulness', 'answer_relevancy', 'context_recall', 'context_precision']].mean()
      display(summary)
      return final_report

    else:
      print("평가 결과 없음")
      return None

## version1

In [4]:
from V1_parent import ParentPipeline, Reference

In [6]:
query = "조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."

pipeline = ParentPipeline(query, vectorstore, llm)
print(Reference().generate_reference(pipeline))

네, 결손금이 발생한 연도에 중소기업에 해당되면 결손금 소급공제 환급신청을 통해 직전 사업연도에 납부한 법인세를 환급받을 수 있습니다.[[3]]

[근거 법령 및 본문]부가가치세법 시행령 91조 ①항 1호: 직전 사업연도에 「조세특례제한법 시행령」 제2조에 따른 중소기업 또는 같은 영 제6조의4제1항에 따른 중견기업에 해당하는 법인(「조세특례제한법」 제6조제3항제2호에 따른 제조업을 주된 사업으로 경영하는 기업에 한정한다)일 것


## version0

In [4]:
from V0_basic import BasicPipeline, Reference

In [5]:
query = "조세특례제한법상 중소기업이 결손금이 발생한 경우 결손금 소급공제 환급신청을 하여 직전 사업연도에 납부한 법인세를 환급받을 수 있다고 규정하고 있는데, 조세특례제한법상 직전 사업연도에는 중소기업에 해당되지 않았는데, 결손금이 발생한 연도에는 중소기업에 해당될 경우에도 환급이 가능한지 답변 부탁드립니다."

pipeline = BasicPipeline(query, llm, vectorstore)
print(Reference().generate_reference(pipeline))

네, 결손금이 발생한 연도에 중소기업에 해당되면 결손금 소급공제 환급신청이 가능하며, 이는 직전 사업연도에 중소기업이 아니었더라도 결손금 발생 시점에 중소기업인 경우에 한정하여 환급이 허용되기 때문입니다 [[0], [3]].

[근거 법령 및 본문]부가가치세법 시행령 91조 ①항 2호 가목: 직전 사업연도에 「조세특례제한법 시행령」 제2조에 따른 중소기업인 경우: 직전 사업연도에 공급한 재화 또는 용역의 공급가액의 합계액에서 수출액이 차지하는 비율이 30퍼센트 이상이거나 수출액이 50억원 이상일 것
부가가치세법 시행령 91조 ①항 1호: 직전 사업연도에 「조세특례제한법 시행령」 제2조에 따른 중소기업 또는 같은 영 제6조의4제1항에 따른 중견기업에 해당하는 법인(「조세특례제한법」 제6조제3항제2호에 따른 제조업을 주된 사업으로 경영하는 기업에 한정한다)일 것


In [4]:
from V3_ensemble import Pipeline
from V2_multiquery import MultiQueryPipeline
from V1_parent import ParentPipeline
from V0_basic import BasicPipeline

In [5]:
import nest_asyncio
import logging
from datasets import Dataset
from google.colab import userdata

try:
  from ragas import evaluate
except ImportError:
  !pip install ragas
  from ragas import evaluate

try:
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
except ImportError:
  !pip install ragas
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

import pandas as pd
import json
from ragas.run_config import RunConfig

nest_asyncio.apply()

logging.getLogger("ragas").setLevel(logging.ERROR)
logging.getLogger("openai").setLevel(logging.ERROR)

def evaluation(version:str, n_tries: int = 3):
  file_path = ['/content/drive/MyDrive/rag/datasets/test_법인세법.json', '/content/drive/MyDrive/rag/datasets/test_vat.json', '/content/drive/MyDrive/rag/datasets/test_소득세.json', '/content/drive/MyDrive/rag/datasets/test_양도소득세.json']
  category = ['법인세법', 'vat', '소득세', '양도소득세']

  final_summary = []
  best_perf = []

  llm_ragas = LangchainLLMWrapper(llm)
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)

  run_config = RunConfig(max_retries=3, timeout=180, max_workers=1)

  for path, law_name in zip(file_path, category):

    try:
      with open(path, 'r', encoding='utf-8') as f:
        test_data = json.load(f)

      try_results = []

      for i in range(n_tries):
        print(f"{law_name}: {i+1}번째 evaluation")

        answers = []
        contexts = []
        questions = []
        ground_truths = []

        for item in test_data:
          query = item['query']
          try:
            if version == 'v3':
              pipeline = Pipeline(query, llm, vectorstore)
            elif version == 'v2':
              pipeline = MultiQueryPipeline(query, llm, vectorstore)
            elif version == 'v1':
              pipeline = ParentPipeline(query, vectorstore, llm)
            elif version == 'v0':
              pipeline = BasicPipeline(query, llm, vectorstore)
            else:
              raise ValueError("v0, v1, v2, v3 중 선택하세요.")

            answer, _, retrieved_docs = pipeline.generate_answer()
            answers.append(answer)
            contexts.append([doc.page_content for doc in retrieved_docs])
            questions.append(query)
            ground_truths.append(item['answer'])

          except Exception as e:
            print(f"평가 중 에러 발생: {e}")
            continue

        data = {
          "user_input": questions,
          "response": answers,
          "retrieved_contexts": contexts,
          "reference": ground_truths
        }

        dataset = Dataset.from_dict(data)

        result = evaluate(
            dataset=dataset,
            metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
            llm=llm_ragas,
            embeddings=embedding_ragas,
            run_config=run_config
        )

        df_try = result.to_pandas()
        df_try['try_no'] = i + 1
        try_results.append(df_try)

      if try_results:
        all_tries_df = pd.concat(try_results, ignore_index=True)
        cols = ['faithfulness', 'answer_relevancy', 'context_recall', 'context_precision']

        avg = all_tries_df[cols].mean()
        print(f"{law_name} {n_tries}회 평균")
        print(avg)

        all_tries_df['sum'] = all_tries_df[cols].sum(axis=1)
        try_scores = all_tries_df.groupby('try_no')['sum'].mean()
        best_try_idx = int(try_scores.idxmax())
        best_try = all_tries_df[all_tries_df['try_no'].values == best_try_idx].copy()
        best_try['law_name'] = law_name

        print(f"best performance: {best_try_idx}번째")
        display(best_try)
        best_perf.append(best_try)

        summary = avg.to_frame().T
        summary['law_name'] = law_name
        final_summary.append(summary)

    except Exception as e:
      print(f"{law_name} 평가 중 오류 발생: {e}")


  if final_summary:
      final_report = pd.concat(final_summary, ignore_index=True)
      print("-"*50)
      print("최종 평가 결과 요약")
      display(final_report.set_index('law_name'))
      return final_report

  return None


/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/tmp/ipython-input-1578671129.py:13: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
/tmp/ipython-input-1578671129.py:13: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics

In [6]:
eval0 = evaluation('v0')

/tmp/ipython-input-1578671129.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1578671129.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


법인세법: 1번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.453333
answer_relevancy     0.579804
context_recall       1.000000
context_precision    0.750791
dtype: float64
best performance: 2번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
5,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,[⑦법 제55조의2제1항제1호의 규정을 적용함에 있어서 예약매출에 의하여 토지 등을...,"공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우, 양도세 ...","현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.000000,0.549197,1.0,0.383333,2,2.932531,양도소득세
6,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,"[제83조(양도소득세가 중과되는 1세대 2주택에 관한 특례의 요건), 제94조(양도...",양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 제83조의 규정에 따라 1세...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,0.000000,0.971684,1.0,1.000000,2,2.971684,양도소득세
7,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,"[1. 양도차손이 발생한 자산과 같은 세율을 적용받는 자산의 양도소득, 2. 양도차...","주택 2채를 동시에 양도하는 경우, 제55조제1항에 따른 세율에 20% 또는 30%...",1세대 1주택 비과세 요건(보유기간 2년 이상*)을 충족한 2개 이상의 주택을 같은...,0.666667,0.734813,1.0,0.948413,2,3.349893,양도소득세
8,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 2. 증여자가 직접 양도하는 경우로 보아 ...","증여자가 사망한 후 배우자가 양도하는 경우, 비과세 여부는 제공된 자료에서 명확히 ...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.666667,0.000000,1.0,0.854167,2,2.520833,양도소득세
9,국민주택채권 등 매각차손 필요경비 산입 여부는?,[① 법 제31조제1항에 따라 필요경비에 산입하는 보험차익 상당액은 일시상각충당금으...,국민주택채권 등 매각차손은 필요경비에 산입되지 않습니다.,소득세법」제94조 제1항 제1호의 자산(토지․건물)을 취득함에 있어서 법령 등의 규...,0.000000,0.836059,1.0,0.596190,2,2.432249,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.555556,0.588001,0.977778,0.825057
vat,0.441667,0.577041,1.000000,0.676745
소득세,0.538889,0.531130,0.732937,0.624016
양도소득세,0.453333,0.579804,1.000000,0.750791


In [7]:
eval1 = evaluation('v1')

법인세법: 1번째 evaluation


/tmp/ipython-input-1578671129.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1578671129.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.227778
answer_relevancy     0.601601
context_recall       1.000000
context_precision    0.677477
dtype: float64
best performance: 3번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
10,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,[⑦법 제55조의2제1항제1호의 규정을 적용함에 있어서 예약매출에 의하여 토지 등을...,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에는 양도세...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",0.50,0.911726,1.0,0.327778,3,2.739504,양도소득세
11,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,"[제83조(양도소득세가 중과되는 1세대 2주택에 관한 특례의 요건), 제94조(양도...",양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 세법상 가족으로서 배우자와 ...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,0.00,0.971698,1.0,1.000000,3,2.971698,양도소득세
12,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,"[1. 양도차손이 발생한 자산과 같은 세율을 적용받는 자산의 양도소득, 2. 양도차...","주택 2채를 동시에 양도한 경우, 각각의 양도소득에 대해 별도로 양도소득세가 과세되...",1세대 1주택 비과세 요건(보유기간 2년 이상*)을 충족한 2개 이상의 주택을 같은...,0.00,0.765752,1.0,0.842679,3,2.608432,양도소득세
13,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,"[2. 배우자가 사망하거나 이혼한 경우, 2. 증여자가 직접 양도하는 경우로 보아 ...","배우자가 사망 후 양도하는 경우, 해당 양도는 증여로 간주되지 않으므로 비과세 여부...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.25,0.000000,1.0,0.680158,3,1.930158,양도소득세
14,국민주택채권 등 매각차손 필요경비 산입 여부는?,[① 법 제31조제1항에 따라 필요경비에 산입하는 보험차익 상당액은 일시상각충당금으...,제공된 자료에서 국민주택채권 등 매각차손 필요경비 산입 여부에 관한 구체적인 내용은...,소득세법」제94조 제1항 제1호의 자산(토지․건물)을 취득함에 있어서 법령 등의 규...,1.00,0.000000,1.0,0.620297,3,2.620297,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.433333,0.568665,0.950000,0.787904
vat,0.504598,0.609381,1.000000,0.650995
소득세,0.428571,0.505217,0.733333,0.520651
양도소득세,0.227778,0.601601,1.000000,0.677477


In [8]:
eval2 = evaluation('v2')

법인세법: 1번째 evaluation


/tmp/ipython-input-1578671129.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1578671129.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

양도소득세 3회 평균
faithfulness         0.744444
answer_relevancy     0.707000
context_recall       1.000000
context_precision    0.875164
dtype: float64
best performance: 2번째


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision,try_no,sum,law_name
5,공동사업을 경영할 것을 약정하는 계약에 의해 토지 등을 현물출자하는 경우에 양도세가...,"[제47조의2(현물출자 시 과세특례) 제47조의2(현물출자 시 과세특례), 2. 적...",공동사업을 경영할 것을 약정하는 계약에 의한 토지 등의 현물출자는 양도세 과세 대상...,"현물출자란 금전 이외의 현물, 즉 부동산, 채권, 특허권, 영업권 등을 출자하는 것...",1.0,0.823214,1.0,0.876190,2,3.699405,양도소득세
6,양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는?,"[제53조(생계를 같이 하는 부양가족의 범위와 그 판정시기), 제167조의10(양도...",양도소득세에서 1세대 판단시 기준이 되는 가족의 범위는 생계를 같이 하는 부양가족의...,양도소득세와 관련된 1세대 판단시 가족이라 함은 거주자와 그 배우자의 직계존비속(그...,1.0,0.971235,1.0,1.000000,2,3.971235,양도소득세
7,주택 2채를 동시에(같은 날) 양도하였는데 양도소득세는 어떻게 과세됩니까?,[1. 납부할 양도소득세 계산식\n \n 거주주택 양도 당시 해당 임대주택 또는 어...,두 주택의 양도소득세는 각각의 양도소득에 대해 별도로 계산하여 높은 세율을 적용한 ...,1세대 1주택 비과세 요건(보유기간 2년 이상*)을 충족한 2개 이상의 주택을 같은...,1.0,0.613975,1.0,1.000000,2,3.613975,양도소득세
8,증여자가 사망한 후 배우자가 양도하는 경우 비과세 여부?,[1. 거주자가 그 배우자 또는 직계존비속으로부터 증여받은 자산에 대한 증여세 산출...,"증여자가 사망한 후 배우자가 양도하는 경우, 증여세 비과세 여부는 해당 증여가 상속...",소득세법 제89조 제1항 제3호의 1세대1주택 비과세 규정을 적용함에 있어서 배우자...,0.0,0.918737,1.0,0.638889,2,2.557626,양도소득세
9,국민주택채권 등 매각차손 필요경비 산입 여부는?,[가. [양도가액-(기존건물과 그 부수토지의 평가액-지급받은 청산금)-법 제97조 ...,국민주택채권 및 토지개발채권을 만기 전에 양도하여 발생하는 매각차손은 필요경비로 산...,소득세법」제94조 제1항 제1호의 자산(토지․건물)을 취득함에 있어서 법령 등의 규...,1.0,0.646136,1.0,0.810000,2,3.456136,양도소득세


--------------------------------------------------
최종 평가 결과 요약


,faithfulness,answer_relevancy,context_recall,context_precision
law_name,,,,
법인세법,0.655000,0.523787,0.988889,0.938458
vat,0.562778,0.655901,0.966667,0.834712
소득세,0.516667,0.487365,0.788889,0.680380
양도소득세,0.744444,0.707000,1.000000,0.875164


In [ ]:
eval3 = evaluation('v3')

법인세법: 1번째 evaluation


/tmp/ipython-input-1578671129.py:37: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm_ragas = LangchainLLMWrapper(llm)
/tmp/ipython-input-1578671129.py:38: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embedding_ragas = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

법인세법: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

소득세: 2번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

소득세: 3번째 evaluation


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]